# Assignment 2 - DS4Biz Y63
## TextScraping_Classification
***

### Team Detail
**Team Name**: sompinandsomshine   
***
### *Student 1*
**Student ID**: 61070278   
**Student Full Name**: นายกิตติภณ สุรุ่งเรืองสกุล
***
### *Student 2*
**Student ID**: 61070330   
**Student Full Name**: นางสาวอิงฟ้า ภูติวรนาถ
***

#### link: https://quotes.toscrape.com/

# Data Collection

In [1]:
import requests
#https://requests.readthedocs.io/en/master/
import bs4
#https://www.crummy.com/software/BeautifulSoup/bs4/doc/

import pandas as pd

In [2]:
response = requests.get('https://quotes.toscrape.com/page/1/')
print(response)
print(type(response))
html_page = bs4.BeautifulSoup(response.content, 'html.parser')
# print(html_page)

<Response [200]>
<class 'requests.models.Response'>


In [3]:
list_page = []
cur = 1
while 1:
    url = 'https://quotes.toscrape.com/page/'+str(cur)+'/'
    response = requests.get(url)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')
    selector = 'body > div > div:nth-child(2) > div.col-md-8'
    tag = html_page.select_one(selector)
    if 'No quotes found!' in tag.text:
        break
    else:
        list_page.append(url)
        cur += 1


In [4]:
list_page

['https://quotes.toscrape.com/page/1/',
 'https://quotes.toscrape.com/page/2/',
 'https://quotes.toscrape.com/page/3/',
 'https://quotes.toscrape.com/page/4/',
 'https://quotes.toscrape.com/page/5/',
 'https://quotes.toscrape.com/page/6/',
 'https://quotes.toscrape.com/page/7/',
 'https://quotes.toscrape.com/page/8/',
 'https://quotes.toscrape.com/page/9/',
 'https://quotes.toscrape.com/page/10/']

In [33]:
df = pd.DataFrame(columns=['content','author','tags', 'link_author'])
df

,content,author,tags,link_author


In [34]:
# Content
def extract_content(html_page):
    selector = 'div.col-md-8 > div > span.text'
    tags = html_page.select(selector)
    content = []
    
    for tag in tags:
        content.append(tag.text.strip().replace('“', '').replace('”', ''))
        
    return content

In [35]:
# Author
def extract_author(html_page):
    selector = 'div.col-md-8 > div > span > small'
    tags = html_page.select(selector)
    author = []
    
    for tag in tags:
        author.append(tag.text.strip().replace('-', ' '))
        
    return author

In [36]:
# tags
def extract_tags(html_page):
    selector = 'div.col-md-8 > div > div > meta'
    tags = html_page.select(selector)
    quote_tags = []
    
    for tag in tags:
#         print(tag['content'])
        quote_tags.append(tag['content'])
        
    return quote_tags

In [37]:
# link_author
def extract_link_author(html_page):
    selector = 'div.col-md-8 > div > span > a'
    tags = html_page.select(selector)
    link_author = []
    
    for tag in tags:
#         print(tag['content'])
        link_author.append('https://quotes.toscrape.com/author/' + tag['href'].split('/')[-1] + '/')
        
    return link_author

In [38]:
# response = requests.get('https://quotes.toscrape.com/page/1/')
# html_page = bs4.BeautifulSoup(response.content, 'html.parser')

# extract_content(html_page)

In [39]:
def extract_quotes(url):
    response = requests.get(url)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')

    content = extract_content(html_page)
    author = extract_author(html_page)
    tags = extract_tags(html_page)
    link_author = extract_link_author(html_page)
    
    Alltags = {'content':content,'author':author,'tags':tags,'link_author':link_author}
    result = pd.DataFrame(Alltags)
    return result

In [40]:
for url in list_page:
    result = extract_quotes(url)
    df = df.append(result, ignore_index=True)
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [41]:
df.isna().sum()

content        0
author         0
tags           0
link_author    0
dtype: int64

## author

In [42]:
# df_author = pd.DataFrame(columns=['Born_date','Born_location','Author_description'])
# df_author

In [43]:
# df['Link_author'].iloc[0]

In [44]:
# author born date
def born_date(html_page):
    born_date = html_page.findAll("span", {"class": "author-born-date"})
    born_date = born_date[0].text.replace(',', '').replace(' ', '-')
    return born_date

In [45]:
# author born location
def born_location(html_page):
    born_location = html_page.findAll("span", {"class": "author-born-location"})
    born_location = born_location[0].text.replace(',', '|')
    return born_location

In [46]:
# author Description
def description(html_page):
    description = html_page.findAll("div", {"class": "author-description"})
    description = description[0].text.strip()
    return description

In [47]:
# author Description
def author_name(html_page):
    author_name = html_page.findAll("h3", {"class": "author-title"})
    author_name = author_name[0].text.strip().replace('-', ' ')
    return author_name

In [48]:
# def extract_author(url):
#     response = requests.get(url)
#     html_page = bs4.BeautifulSoup(response.content, 'html.parser')

#     date = born_date(html_page)
#     location = born_location(html_page)
#     description_text = description(html_page)
    
#     Alldetail = {'Born_date': date, 'Born_location': location, 'Author_description':description_text}
#     result = pd.DataFrame(Alldetail)
#     return result

In [49]:
list_born_date = []
list_born_location = []
list_description = []
list_author_name = []

for link_author in list(df['link_author'].unique()):
    
#     response = requests.get('https://quotes.toscrape.com/author/Albert-Einstein/')
    response = requests.get(link_author)
    html_page = bs4.BeautifulSoup(response.content, 'html.parser')
    
    list_born_date.append(born_date(html_page))
    list_born_location.append(born_location(html_page))
    list_description.append(description(html_page))
    list_author_name.append(author_name(html_page))

In [50]:
Alldetail = {'author': list_author_name, 'born_date': list_born_date, 'born_location': list_born_location, 'author_description':list_description}
author_df = pd.DataFrame(Alldetail)

In [51]:
author_df.head()

,author,born_date,born_location,author_description
0,Albert Einstein,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,July-31-1965,in Yate| South Gloucestershire| England| The U...,See also: Robert GalbraithAlthough she writes ...
2,Jane Austen,December-16-1775,in Steventon Rectory| Hampshire| The United Ki...,Jane Austen was an English novelist whose work...
3,Marilyn Monroe,June-01-1926,in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
4,André Gide,November-22-1869,in Paris| France,André Paul Guillaume Gide was a French author ...


In [52]:
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [53]:
# df.set_index('Author').join(other.set_index('key'))

In [54]:
# result['Author'].value_counts()

In [55]:
content_author_df = pd.merge(df, author_df, on='author', how='left')
content_author_df.head()

,content,author,tags,link_author,born_date,born_location,author_description
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/,July-31-1965,in Yate| South Gloucestershire| England| The U...,See also: Robert GalbraithAlthough she writes ...
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...,March-14-1879,in Ulm| Germany,"In 1879, Albert Einstein was born in Ulm, Germ..."
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/,December-16-1775,in Steventon Rectory| Hampshire| The United Ki...,Jane Austen was an English novelist whose work...
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...,June-01-1926,in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...


## save DataFarame to csv
เพื่อให้ง่ายต่อการใช้ในภายหลัง
- content.csv รายละเอียดของ quote
- author.csv รายละเอียดของผู้เขียน
- content_author.csv รายละเอียดของ quote และผู้เขียน

In [56]:
df.to_csv('content.csv', index=False)
author_df.to_csv('author.csv', index=False)
content_author_df.to_csv('content_author.csv', index=False)

In [57]:
df = pd.read_csv('content.csv')
df.head()

,content,author,tags,link_author
0,The world as we have created it is a process o...,Albert Einstein,"change,deep-thoughts,thinking,world",https://quotes.toscrape.com/author/Albert-Eins...
1,"It is our choices, Harry, that show what we tr...",J.K. Rowling,"abilities,choices",https://quotes.toscrape.com/author/J-K-Rowling/
2,There are only two ways to live your life. One...,Albert Einstein,"inspirational,life,live,miracle,miracles",https://quotes.toscrape.com/author/Albert-Eins...
3,"The person, be it gentleman or lady, who has n...",Jane Austen,"aliteracy,books,classic,humor",https://quotes.toscrape.com/author/Jane-Austen/
4,"Imperfection is beauty, madness is genius and ...",Marilyn Monroe,"be-yourself,inspirational",https://quotes.toscrape.com/author/Marilyn-Mon...


In [58]:
df.isna().sum()

content        0
author         0
tags           3
link_author    0
dtype: int64

In [59]:
df[df['tags'].isna()]

,content,author,tags,link_author
27,It is impossible to live without failing at so...,J.K. Rowling,NaN,https://quotes.toscrape.com/author/J-K-Rowling/
42,You believe lies so you eventually learn to tr...,Marilyn Monroe,NaN,https://quotes.toscrape.com/author/Marilyn-Mon...
78,The question isn't who is going to let me; it'...,Ayn Rand,NaN,https://quotes.toscrape.com/author/Ayn-Rand/


### บันทึก tags
folder target

In [60]:
f = open("target/tags.txt", "a")

for i in list(df['tags']):
    f.write(str(i) + '\n')

f.close()